In [ ]:
import numpy as np
import tensorflow as tf

In [ ]:
f = open("input1.txt", "r")
if f.mode == "r":
    corpus = f.read()
    print(corpus)
#corpus = 'he is the man . the man is happy . she is the happy women'

In [ ]:
words = []
for word in corpus.split():
    if word != '.': 
        words.append(word)

In [ ]:
words = set(words)

In [ ]:
words

In [ ]:
word2int = {}
int2word = {}
vocab_size = len(words)

for i,word in enumerate(words):
    word2int[word] = i
    int2word[i] = word

In [ ]:
#print(word2int[''])

In [ ]:
#print(int2word[4])

In [ ]:
raw_sentences = corpus.split('.')
sentences = []
for sentence in raw_sentences:
    sentences.append(sentence.split())

In [ ]:
raw_sentences

In [ ]:
print(sentences)

In [ ]:
data = []
WINDOW_SIZE = 2

def to_one_hot(data_point_index, vocab_size):
    temp = np.zeros(vocab_size)
    temp[data_point_index] = 1
    return temp

In [ ]:
for sentence in sentences:
    for word_index, word in enumerate(sentence):
        for nb_word in sentence[max(word_index - WINDOW_SIZE, 0) : min(word_index + WINDOW_SIZE, len(sentence)) + 1] : 
            if nb_word != word:
                data.append([word, nb_word])

In [ ]:
data

In [ ]:
x_train = [] 
y_train = [] 
for data_word in data:
    x_train.append(to_one_hot(word2int[ data_word[0] ], vocab_size))
    y_train.append(to_one_hot(word2int[ data_word[1] ], vocab_size))

In [ ]:
x_train = np.asarray(x_train)
y_train = np.asarray(y_train)

In [ ]:
print(x_train.shape, y_train.shape)

In [ ]:
x = tf.placeholder(tf.float32, shape=(None, vocab_size))
y_label = tf.placeholder(tf.float32, shape=(None, vocab_size))

In [ ]:
EMBEDDING_DIM = 5 
W1 = tf.Variable(tf.random_normal([vocab_size, EMBEDDING_DIM]))
b1 = tf.Variable(tf.random_normal([EMBEDDING_DIM]))
hidden_representation = tf.add(tf.matmul(x,W1), b1)

In [ ]:
W2 = tf.Variable(tf.random_normal([EMBEDDING_DIM, vocab_size]))
b2 = tf.Variable(tf.random_normal([vocab_size]))
prediction = tf.nn.softmax(tf.add( tf.matmul(hidden_representation, W2), b2))

In [ ]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init) 

# define the loss function:
cross_entropy_loss = tf.reduce_mean(-tf.reduce_sum(y_label * tf.log(prediction), reduction_indices=[1]))
# define the training step:
train_step = tf.train.GradientDescentOptimizer(0.1).minimize(cross_entropy_loss)
n_iters = 10000
# train for n_iter iterations
for _ in range(n_iters):
    sess.run(train_step, feed_dict={x: x_train, y_label: y_train})
    print('loss is : ', sess.run(cross_entropy_loss, feed_dict={x: x_train, y_label: y_train}))

In [ ]:
print(sess.run(W1))
print(sess.run(b1))

In [ ]:
vectors = sess.run(W1 + b1)

In [ ]:
vectors

In [ ]:
#print(vectors[word2int['kid']])

In [ ]:
def euclidean_dist(vec1, vec2):
    return np.sqrt(np.sum((vec1-vec2)**2))

def find_closest(word_index, vectors):
    min_dist = 10000 # to act like positive infinity
    min_index = -1
    query_vector = vectors[word_index]
    for index, vector in enumerate(vectors):
        if euclidean_dist(vector, query_vector) < min_dist and not np.array_equal(vector, query_vector):
            min_dist = euclidean_dist(vector, query_vector)
            min_index = index
    return min_index

In [ ]:
def cosine_sim(vec1, vec2):
    in_pro = np.sum(vec1*vec2)
    mag_vec1 = np.sqrt(np.sum(vec1**2))
    mag_vec2 = np.sqrt(np.sum(vec2**2))
    ans = in_pro/(mag_vec1*mag_vec2)
    return ans

vec_fix = vectors[6]
for index, vector in enumerate(vectors):
    cos = {}
    temp = cosine_sim(vec_fix, vectors[index])
    cos[index] = temp
    print(cos)

In [ ]:
corpus

In [ ]:
words

In [ ]:
vectors

In [ ]:
print(vectors[word2int['have']])

In [ ]:
print(vectors[word2int['has']])

In [ ]:
print(int2word[find_closest(word2int['have'], vectors)])